In [1]:
import numpy as np
import pandas as pd
import csv
import random
import math

# Инициализация

In [2]:
# количество признаков входного слоя
feature_count = 4

# Считывание данных из файла и их подготовка

In [3]:
df = pd.read_csv('data/titanik.csv', sep=',')

drop_column = ['SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin']

# подготовленный датасет
train_data = []
# датасет для тестирования
test_data = []

classes = df['Survived'].unique()

df['Sex'] = df['Sex'].replace(['male','female'], [0, 1])
df['Embarked'] = df['Embarked'].fillna(('S'))
df['Embarked'] = df['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
df['Pclass'] = df['Pclass'].fillna(np.mean(df['Pclass']))
df['Age'] = df['Age'].fillna(df['Age'].median())

for i in range(len(df)) : 

    features = np.zeros((feature_count))
    features[0] = df.loc[i, 'Sex']
    features[1] = df.loc[i, 'Embarked']
    features[2] = df.loc[i, 'Pclass']
    features[3] = df.loc[i, 'Age']
    
    
    # Нормализация входного вектора
    norm_features = np.zeros((feature_count))
    for j in range(len(features)):
        sum = 0 
        for k in range(len(features)):
            sum += features[k] ** 2
        norm_features[j] = features[j] / math.sqrt(sum)
    
    temp_data = dict(name = df.loc[i, 'Name'], 
                     features = norm_features, 
                     result = np.where(classes == df.loc[i, "Survived"])[0].item(0))
    
    if (len(train_data) / len(df)) < 0.9:
        train_data.append(temp_data)
    else:
        test_data.append(temp_data)

# Обучение

## Кохонен

In [4]:
def getKohonenWinnerIndex(x, neuron_counts, w, s):
    result = 0
    max_net = -1
    for i in range(neuron_counts):
        net = 0
        for j in range(len(x)):
            net += x[j] * w[j][i] * s[i]
        if (net > max_net):
            result = i
            max_net = net
    return result

In [5]:
# скорость обучения слоя Кохонена
theta = 0.7
# скорость обучения слоя Гроссберга
beta = 0.2
#количество нейронов выходного слоя (слоя Гроссберга) = количество проффессий 
grossberg_neuron_counts = len(classes)
# количество нейронов кохонена
kohonen_neuron_count = 16
# штрафы кластеров
s_w = np.ones((kohonen_neuron_count))

#инициализация начальных весов слоя Кохонена kohonen_w[x][y]
# x - индекс нейрона входного слоя
# y - индекс нейрона слоя Кохонена
kohonen_w = np.zeros((feature_count, kohonen_neuron_count))
for i in range(feature_count):
    for j in range(kohonen_neuron_count):
        kohonen_w[i, j] = random.uniform(0.01, 0.99)
        
# Инициализация начальных весов слоя Гроссберга grossberg_w[x][y]
# x - индекс нейрона слоя Кохонена
# y - индекс нейрона слоя Гроссберга
grossberg_w = np.zeros((kohonen_neuron_count, grossberg_neuron_counts))
for i in range(kohonen_neuron_count):
    for j in range(grossberg_neuron_counts):
        grossberg_w[i, j] = random.uniform(0, 1)
        
# Обучение на подготовленном датасете
for man in train_data:
    
    # Определение нейрона победителя
    winner_index = getKohonenWinnerIndex(man['features'], kohonen_neuron_count, kohonen_w, s_w)
    s_w[winner_index] *= 0.9
    
    # Корректировка весов нейрона победителя на слое Кохонена
    for i in range(feature_count):
        kohonen_w[i][winner_index] += theta * (man['features'][i] - kohonen_w[i][winner_index])
    
    # Корректировка весов для нейронов слоя Гроссберга, соединенных с нейроном победителем слоя Кохонена
    for i in range(grossberg_neuron_counts):
        if i == man['result']:
            grossberg_w[winner_index][i] += beta * (1 - grossberg_w[winner_index][i])
        else:
            grossberg_w[winner_index][i] += beta * (0 - grossberg_w[winner_index][i])
    
    # Изменение скорости обучения
    theta, beta = theta * 0.9, beta * 0.9  

# Тестирование

In [6]:
accuracy = 0
for man in test_data:
    
    print('Класс: ' + str(classes[man['result']]))
    print('ФИО: ' + man['name'])
    print(man['features'])
    print('Результаты нейросети:')
    
    # Определение нейрона победителя
    winner_index = getKohonenWinnerIndex(man['features'], kohonen_neuron_count, kohonen_w, s_w)
    best_result = 0.0
    best_index = 0

    for i in range(grossberg_neuron_counts):
        if (grossberg_w[winner_index][i] > best_result):
            best_result = grossberg_w[winner_index][i]
            best_indexe = i
        print(str(classes[i]) + ': ' + str(grossberg_w[winner_index][i] * 100) + '%')
    
    if (man['result'] == best_index):
        accuracy += 1
    print('--------------------------------------------------')
print ('Точность: ' + str((accuracy / len(test_data)) * 100) + '%')

Класс: 1
ФИО: Carter, Master. William Thornton II
[0.         0.         0.09053575 0.99589321]
Результаты нейросети:
0: 54.364457316036194%
1: 33.23374753048246%
--------------------------------------------------
Класс: 1
ФИО: Thomas, Master. Assad Alexander
[0.         0.313475   0.94042501 0.1316595 ]
Результаты нейросети:
0: 94.15050092953553%
1: 41.6301203838632%
--------------------------------------------------
Класс: 1
ФИО: Hedman, Mr. Oskar Arvid
[0.         0.         0.11043153 0.99388373]
Результаты нейросети:
0: 54.364457316036194%
1: 33.23374753048246%
--------------------------------------------------
Класс: 0
ФИО: Johansson, Mr. Karl Johan
[0.         0.         0.09632419 0.99535001]
Результаты нейросети:
0: 54.364457316036194%
1: 33.23374753048246%
--------------------------------------------------
Класс: 0
ФИО: Andrews, Mr. Thomas Jr
[0.         0.         0.0256326  0.99967143]
Результаты нейросети:
0: 54.364457316036194%
1: 33.23374753048246%
----------------------